# Kubernetes, Db2 Operator, and Db2U Installation

The Db2 Click to Containerize lab requires the use of an OpenShift, Kubernetes, or OpenShift environment to demonstrate the ability to containerize a Db2 database. This section will take the user through the install of the Db2 operator and Db2U Pod. Similar steps would be done in an OpenShift environment.

# Db2U Installation

This lab is using a Micro Kubernetes environment to demonstrate the use of Db2 in a K8s cluster and for a target of the Db2 Shift command. More details about MicroK8s can be found at https://microk8s.io/. Kubernetes clusters are supported by all major cloud vendors, including AWS (EKS), Microsoft (AKS), Google (GKS) and IBM (IKS). 
MicroK8s is a minimal production Kubernetes. It provides the functionality of core Kubernetes components in a small footprint. All the essential services have been installed to allow for the deployment of Db2U.
The MicroK8s cluster has been populated with the following services:

* Domain Name Service
* Kubernetes Dashboard
* Host Path Storage
* Role-based access control
* Lifecycle manager

## Kubernetes Commands
The MicroK8s server is running on machine number 3 (10.0.0.3). The Kubernetes server has already been installed for you. What we need to do is install the Db2U Operator and then deploy a Db2U Pod.

The Kubernetes command shell is `kubectl`. When running in the MicroK8s environment, the `kubectl` command must be prefixed with the `microk8s` command. In order to make life easier, the `kubeectl` client has been installed on all servers in the cluster.

### Namespaces
In the OpenShift environment you work with Projects instead of Namespaces. Usually, you would switch to a project and then all your OC commands would work against that project. The `kubectl` command doesn't switch to a namespace. Instead, the `kubectl` command can be set with a default context or you can supply the namespace as part of the command. In the lab we will be using the `-n` (namespace) option to specify which namespace we are referring to. 

### List Existing Pods
To list the pods that are currently running in a namespace, use the following command:
`kubectl get pods -n db2u`

Since we haven't create a `db2u` namespace yet, we can list all pods by using the `-A` flag.

In [ ]:
!kubectl get pods -A

### List Services
To determine which services are currently running the cluster, use the following command (note the use of the `-A` flag to show all services).

In [ ]:
!kubectl get svc -A

### Execute a Command in a Pod
To execute a command within the Db2U pod, you must use the following command:
```bash
kubectl exec --stdin --tty c-demo-db2u-0 -n db2u -- /bin/bash`
```
This is harder to remember that the `oc ssh` command!

### Credentials
In an OpenShift environment you will either a userid or a token that you will use to authenticate into the cluster. For the Kubernetes environment we will be using the config file generated for the server. You will not have to create this file since it was done for you and copied onto the other servers in order to do the lab.

For reference, the microk8s command can be used to generate a configuration file which can be used with the kubectl command to connect to the server.
`microk8s config > config`

The config file that is generated contains all the information required to connect to the microk8s server that is running on this server. The `kubectl` command will search in a hidden directory (`~./kube`) for a file named `config`. If this file is found it will then use it for authenticating against the Kubernetes cluster. Once this file is created, a directory called `./kube` needs to be created and the config file copied into it:
```bash
mkdir ~/.kube
cp config ~/.kube/config
```

For this lab, the config file was already generated and copied to all of the userids that need access to the cluster.

## Installing the Db2U Operator
There are three requirements for installing Db2U on a Kubernetes cluster:
* An Operator Lifecycle Manager (OLM)
* The Db2U Operator being installed
* Storage paths

The Operator Lifecycle Manager (OLM) has already been installed on this cluster. If you are interested in what this provides, see the following article: https://sdk.operatorframework.io/docs/overview/

The Db2U operator relies on the OLM to provide the necessary services required to deploy the Db2U pod. To install the Db2U operator you must be connected to the Kubernetes cluster and have the appropriate credentials.

The following is the YAML file that will be used to create the Db2U Operator. The file has already been created for you and can be found in the Documents directory for the k8s userid (`/home/k8s/Documents/db2uoperator.yaml`). 

```yaml
apiVersion: v1
kapiVersion: v1
kind: Namespace
metadata:
  name: db2u
---
apiVersion: operators.coreos.com/v1alpha1
kind: CatalogSource
metadata:
  name: db2u-catalog
  namespace: olm
spec:
  sourceType: grpc
  image: icr.io/db2u/db2-catalog@sha256:e6ac7746ae2d6d3bb4fbf75144d94e400db887c1702b19e08b9533752d896178
  displayName: Db2u Operators
  publisher: IBM Db2
---
apiVersion: operators.coreos.com/v1alpha2
kind: OperatorGroup
metadata:
  name: db2u-oprator-og
  namespace: db2u
spec:
  targetNamespaces:
  - db2u
---
apiVersion: operators.coreos.com/v1alpha1
kind: Subscription
metadata:
  name: db2u-operator-v2-0-0-sub
  namespace: db2u
spec:
  name: db2u-operator
  source: db2u-catalog
  sourceNamespace: olm
  installPlanApproval: Automatic
```

To create the Db2 Operator, execute the following command.

In [ ]:
!kubectl create -f ~/Documents/db2uoperator.yaml

This command will install the Db2U operator into the cluster. To check the progress of the installation, run the following kubectl command. 

In [ ]:
!kubectl get pods -n db2u

It may take a few minutes before you see that the operator has successfully started (the Ready value needs to be 1/1). 
```
NAME                                  READY   STATUS    RESTARTS   AGE
db2u-operator-manager-c849d76c4-pcd6d 1/1     Running   0          37s
```

You can rerun the `get pods` command multiple times to get the status of the creation of the Db2U operator. You can also add the `-w` flag at the end of the command which will cause the command to continue monitoring the pod creation. When you see that the operating is running, use the stop button to cancel the command.

In [ ]:
!kubectl get pods -n db2u -w

## Installing Db2U
The next step is to install Db2U itself. The default YAML for the Db2U deployment is found below. There are several fields that are key to deploying the Db2U pod properly:
* name: `demo` 
This name is used to identify the Db2U deployment
* namespace: `db2u`
The name space must be the same one that we deployed with the Db2 operator in the previous step
* dbtype: `db2oltp`
Database type lets the operator know if it is an OLTP database or a Warehouse database
* password: `db2oltp`
You will need to remember this if you want to connect to the database remotely
* database: `DB2OLTP`
This is the name of the empty database that is generated inside the POD
* cpu: `'4'`, memory: `'8Gi'`
The number of VPCs and memory that is allocated to this Db2U database
* storageClassName: `"microk8s-hostpath"`
The storage that Db2U will be using for this deployment is the native hostpath storage
* storage: `5Gi`
The amount of storage to set aside for the metadata and database. Note that this is not hard enforced in this environment

The YAML file for creating the Db2 database is called `db2create.yaml` and can be found in the Documents directory (`/home/k8s/Documents/db2create.yaml`).
```yaml
apiVersion: db2u.databases.ibm.com/v1
kind: Db2uCluster
metadata:
  name: demo 
  namespace: db2u
spec:
  account:
    privileged: true
  environment:
    dbType: db2oltp
    instance:
      password: db2oltp
    ldap:
      enabled: false
    database:
      name: DB2OLTP
  license:
    accept: true
  podConfig:
    db2u:
      resource:
        db2u:
          limits:
            cpu: '4'
            memory: 8Gi
          requests:
            cpu: '4'
            memory: 8Gi
  size: 1
  version: "11.5.7.0"
  storage:
    - name: meta
      type: "create"
      spec:
        storageClassName: "microk8s-hostpath"
        accessModes:
          - ReadWriteMany
        resources:
          requests:
            storage: 5Gi
    - name: data
      type: "create"
      spec:
        storageClassName: "microk8s-hostpath"
        accessModes:
          - ReadWriteOnce
        resources:
          requests:
            storage: 5Gi
```

The following command will start the installation of the Db2U pod.

In [ ]:
!kubectl apply -f ~/Documents/db2create.yaml

The apply command will start the process of deploying the Db2U container. To monitor the progress of the Db2U creation, use the following command. The addition of the `-w` flag (wait) will show the progress of the creation of the various Db2 objects. Note that it may take several minutes before there is any progress. This is caused by the Db2U code being downloaded to the local server before the installation begins.

In [ ]:
!kubectl get pods -n db2u -w

You should see output similar to the following.
```text
NAME                                    READY   STATUS    	   RESTARTS   AGE
db2u-operator-manager-c849d76c4-pcd6d   1/1     Running             15         12d
c-demo-instdb-dddbc                     0/1     Pending             0          0s
c-demo-instdb-dddbc                     0/1     ContainerCreating   0          0s
c-demo-instdb-dddbc                     1/1     Running             0          2s
c-demo-instdb-dddbc                     0/1     Completed           0          14s
c-demo-etcd-0                           0/1     Pending             0          0s
c-demo-etcd-0                           0/1     ContainerCreating   0          0s
c-demo-etcd-0                           0/1     Running             0          1s
c-demo-etcd-0                           1/1     Running             0          21s
c-demo-db2u-0                           0/1     Pending             0          0s
c-demo-db2u-0                           0/1     Init:0/2            0          1s
c-demo-db2u-0                           0/1     Init:1/2            0          4s
c-demo-db2u-0                           0/1     PodInitializing     0          5s
c-demo-db2u-0                           0/1     Running             0          6s
c-demo-db2u-0                           1/1     Running             0          33s
c-demo-restore-morph-ps2b7              0/1     Pending             0          0s
c-demo-restore-morph-ps2b7              0/1     Init:0/1            0          0s
c-demo-restore-morph-ps2b7              0/1     PodInitializing     0          3s
c-demo-restore-morph-ps2b7              1/1     Running             0          4s
c-demo-restore-morph-ps2b7              0/1     Completed           0          95s
```

When the final `restore-morph job is completed`, we know that the Db2U container is available for the lab. Use the Stop button on the menu bar to cancel the command.

To doublecheck that the instance is ready, use the following command to check the contents of the database.

In [ ]:
%%bash --no-raise-error
kubectl exec -i c-demo-db2u-0 -n db2u -- bash << EOF
cat << EOT > /tmp/db2cmd.db2
CONNECT TO DB2OLTP;
LIST TABLES;
QUIT;
EOT
su - db2inst1 "db2 -tvf /tmp/db2cmd.db2"
EOF

There should not be any tables found in the database! At this point your Db2U database is ready for use in the lab.

#### Credits: IBM 2024, George Baklarz [baklarz@ca.ibm.com]